## Exercícios: LangChain - Conectores
___

* RunnableLambda
* RunnableParallel
* RunnablePassThrough

In [1]:
from langchain.schema.runnable import RunnableLambda

In [2]:
dobrar = RunnableLambda(lambda x: x * 2)
print(dobrar.invoke(2))

4


In [3]:
print(dobrar.invoke(668))

1336


In [4]:
soma_lista = RunnableLambda(lambda lista: sum(lista))
print(soma_lista.invoke([1, 2, 3, 4, 5]))

15


In [5]:
to_upper = RunnableLambda(lambda input_str: input_str.upper())
print(to_upper.invoke("Olá, mundo!"))

OLÁ, MUNDO!


In [6]:
input_data = "Langchain"
print(to_upper.invoke(input_data))

LANGCHAIN


In [7]:
count_char = RunnableLambda(lambda input_chars: len(input_chars))
print(count_char.invoke(input_data))

9


In [8]:
print(count_char.invoke("Olá, mundo!"))

11


`RunnableLambda` encadeado:

**Encadeando `RunnableLambda` conseguimos criar pipelines de transformação.**

In [9]:
# Criando as transformações
to_upper = RunnableLambda(lambda input_str: input_str.upper())

count_chars = RunnableLambda(
    lambda input_chars: len(input_chars)
    )

# Montando pipeline (Encadeando as transformações)
pipeline = to_upper | count_chars

In [10]:
print(pipeline.invoke("Olá, mundo!"))

11


In [11]:
print(pipeline.invoke("Fabio Lima"))

10


### RunnableParallel
____

**`RunnableParallel`**: Executa múltiplos `Runnables` em paralelo. Muito útil para montar dicionários que servirão de entrada para um prompt com vários placeholders.

<h3>Enunciado:<h3>


Crie um `RunnableParallel` que receba uma palavra e produza em paralelo:

A palavra em maiúsculas

A palavra em minúsculas

O tamanho da palavra

Explicação:

O RunnableParallel executa diferentes transformações ao mesmo tempo sobre a mesma entrada, retornando um dicionário com os resultados.


In [12]:
from langchain.schema.runnable import RunnableParallel
import json
from typing import Dict, Any
from langchain_core.runnables import RunnableSerializable

pipeline: RunnableSerializable[str, Dict[str, Any]] = RunnableParallel({
    "upper": to_upper,
    "lower": RunnableLambda(lambda input_str: input_str.lower()),
    "length": count_chars
})#type: ignore

result =pipeline.invoke("Fabio Lima") #type: ignore
print(json.dumps(result, indent=4))


{
    "upper": "FABIO LIMA",
    "lower": "fabio lima",
    "length": 10
}


**`RunnablePassthrough`**: Um componente crucial para gerenciar dados.

        * Sozinho, ele simplesmente passa a entrada adiante.
        * Com **`.assign()`**, ele executa uma cadeia e adiciona
        * seu resultado como uma nova chave no dicionário de entrada, **preservando os dados originais**. Este é o padrão-ouro para passar informações entre etapas de um workflow.

`Enunciado`:

Crie um pipeline que:
1. Receba uma frase:
2. Use `RunnablePassthrough` para passar a frase adiante sem alterar.
3. Conte quantas palavras tem a frase 

Com **`RunnablePassthrough`** é útil quando queremos repassar a entrada intacta dentro de uma pipeline.
Assim, pode se injetar a entrada original em diferentes pontos.


In [13]:
from langchain.schema.runnable import RunnablePassthrough #type: ignore
#* 1. Receba uma frase:
frase: str = "A vida que se leva é a vida que se vive !"

#* 2. Use `RunnablePassthrough` para passar a frase adiante sem alterar.
chain_1 = RunnablePassthrough() #type: ignore

#* 3. Conte quantas palavras tem a frase de entrada: RunnableLambda
count_words = RunnableLambda(lambda input_str: len(input_str.split())) #type: ignore

#* 4. Pipeline:
pipeline = chain_1 | count_words #type: ignore

#* 5. Execute o pipeline:
result = pipeline.invoke(frase) #type: ignore
print(result)

12


#### `RunnablePassthrough().assign()`
___

Demonstração do RunnablePassthrough().assign()
==============================================

Este código complementa o exercício da célula 15 do notebook, demonstrando como
o RunnablePassthrough().assign() executa uma chain e adiciona seu resultado
como uma nova chave no dicionário de entrada, preservando os dados originais.

Este é o padrão-ouro para passar informações entre etapas de um workflow.

**`O input do pipeline de dados deve ser um dicionário de entrada.`**

In [14]:
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
import json


In [15]:
# Nossa frase base para os experimentos
frase = "A vida que se leva é a vida que se vive !"
print("Frase de entrada:", frase)


Frase de entrada: A vida que se leva é a vida que se vive !


In [16]:
# A entrada para o pipeline DEVE ser um dicionário.
# A convenção é usar uma chave como "input" ou "text".
input_dict = {"input": frase}

In [17]:
#* Cria pipeline que adiciona novos campos, mas mantém o original
input = input_dict["input"]
pipeline_basico = RunnablePassthrough().assign(
    num_palavras=RunnableLambda(lambda texto: len(input.split())),
    frase_maiuscula=RunnableLambda(lambda texto: input.upper()),
    num_caracteres=RunnableLambda(lambda texto: len(input))
)

resultado_basico = pipeline_basico.invoke(input_dict)

print("Resultado do pipeline básico:")
print("Frase original:", resultado_basico["input"])
print("Número de palavras:", resultado_basico["num_palavras"])
print("Frase em maiúsculas:", resultado_basico["frase_maiuscula"])
print("Número de caracteres:", resultado_basico["num_caracteres"])


Resultado do pipeline básico:
Frase original: A vida que se leva é a vida que se vive !
Número de palavras: 12
Frase em maiúsculas: A VIDA QUE SE LEVA É A VIDA QUE SE VIVE !
Número de caracteres: 41


In [18]:
print("Chaves disponíveis no resultado:", list(resultado_basico.keys()))
print("Estrutura completa do resultado (JSON):")
print(json.dumps(resultado_basico, indent=2, ensure_ascii=False))


Chaves disponíveis no resultado: ['input', 'num_palavras', 'frase_maiuscula', 'num_caracteres']
Estrutura completa do resultado (JSON):
{
  "input": "A vida que se leva é a vida que se vive !",
  "num_palavras": 12,
  "frase_maiuscula": "A VIDA QUE SE LEVA É A VIDA QUE SE VIVE !",
  "num_caracteres": 41
}


In [19]:
print("Estrutura completa do resultado (JSON): \nResultado da pipeline básica =\n")
print(json.dumps(resultado_basico, indent=4, ensure_ascii=False))

Estrutura completa do resultado (JSON): 
Resultado da pipeline básica =

{
    "input": "A vida que se leva é a vida que se vive !",
    "num_palavras": 12,
    "frase_maiuscula": "A VIDA QUE SE LEVA É A VIDA QUE SE VIVE !",
    "num_caracteres": 41
}


Para o `pipeline avançado` utilizamos as operações do `pipeline básico`

In [20]:
pipeline_avancado = pipeline_basico.assign(
    analise=RunnableLambda(lambda dados: {
        "palavras_por_caractere": round(dados["num_palavras"] / dados["num_caracteres"], 3),
        "tem_exclamacao": "!" in dados["input"],
        "palavras_unicas": len(set(dados["input"].lower().split())),
        "palavra_mais_longa": max(dados["input"].split(), key=len),
    })
)

resultado_avancado = pipeline_avancado.invoke(input_dict)

print("Análise detalhada:")
for chave, valor in resultado_avancado["analise"].items():
    print(f"  {chave}: {valor}")


Análise detalhada:
  palavras_por_caractere: 0.293
  tem_exclamacao: True
  palavras_unicas: 8
  palavra_mais_longa: vida


In [ ]:
pipeline_cascata = (
    RunnablePassthrough()
    .assign(
        estatisticas_basicas=RunnableLambda(lambda dados: {
            "comprimento": len(dados["input"]),
            "palavras": len(dados["input"].split()),
            "caracteres_sem_espaco": len(dados["input"].replace(" ", "")),
        })
    )
    .assign(
        analise_semantica=RunnableLambda(lambda dados: {
            "tem_pontuacao": any(c in dados["input"] for c in "!?.,;:"),
            "densidade_texto": round(
                dados["estatisticas_basicas"]["caracteres_sem_espaco"]
                / dados["estatisticas_basicas"]["comprimento"], 3
            )
        })
    )
    .assign(
        resumo=RunnableLambda(lambda dados: {
            "texto_curto": dados["estatisticas_basicas"]["comprimento"] < 50,
            "denso": dados["analise_semantica"]["densidade_texto"] > 0.7,
            "complexidade": "alta" if dados["estatisticas_basicas"]["palavras"] > 10 else "baixa"
        })
    )
)

resultado_cascata = pipeline_cascata.invoke(input_dict)

print("Resultado do pipeline em cascata:")
print("  Estatísticas básicas:", resultado_cascata["estatisticas_basicas"])
print("  Análise semântica:", resultado_cascata["analise_semantica"])
print("  Resumo:", resultado_cascata["resumo"])

### Caso de uso real: Classificação de textos:
___

In [ ]:
textos_exemplo = [
    "Python é uma linguagem de programação incrível!",
    "A inteligência artificial está revolucionando o mundo.",
    "Olá, como você está hoje?",
    "Este é um texto muito longo que contém muitas palavras e informações detalhadas."
]

In [ ]:
pipeline_classificacao = (
    RunnablePassthrough()
    .assign(
        caracteristicas=RunnableLambda(lambda texto: {
            "comprimento": len(texto),
            "palavras": len(texto.split()),
            "tem_exclamacao": "!" in texto,
            "tem_interrogacao": "?" in texto,
        })
    )
    .assign(
        classificacao=RunnableLambda(lambda dados: {
            "tipo": (
                "exclamativo" if dados["caracteristicas"]["tem_exclamacao"]
                else "interrogativo" if dados["caracteristicas"]["tem_interrogacao"]
                else "declarativo"
            ),
            "complexidade": (
                "alta" if dados["caracteristicas"]["palavras"] > 15
                else "média" if dados["caracteristicas"]["palavras"] > 8
                else "baixa"
            )
        })
    )
)

for i, texto in enumerate(textos_exemplo, 1):
    resultado = pipeline_classificacao.invoke(texto)
    print(f"\nTexto {i}: '{texto}'")
    print("  Características:", resultado["caracteristicas"])
    print("  Classificação:", resultado["classificacao"])

In [ ]:
print("\n" + "=" * 60)
print("RESUMO: Vantagens do RunnablePassthrough().assign()")
print("=" * 60)
print("✅ Mantém os dados originais intactos (input)")
print("✅ Permite adicionar novos resultados incrementalmente")
print("✅ Facilita a criação de pipelines em múltiplas etapas")
print("✅ Ajuda no debugging e manutenção")
print("✅ Ideal para casos reais como classificação e análise de texto")
